# TUT Acoustic Scenes Dataset

Having a look at the dataset used to train the acoustic scene classifier

In [27]:
import os
import pandas as pd

TRAINING_DIR = '../data/scenes_training_set'
VALIDATION_DIR = '../data/scenes_validation_set'

In [28]:
# Look at training data
def load_meta_data(path):
    meta_path = os.path.join(path, 'meta.txt')
    with open(meta_path, 'r') as f:
        meta_text = f.read()

    data = []
    for line in meta_text.split('\n'):
        if not line:
            continue
        filename, label = line.split("\t")
        filename_cleaned = filename.replace("audio/", "")
        data.append({"filename": filename_cleaned, "label": label})

    return pd.DataFrame(data=data)

training_meta_df = load_meta_data(TRAINING_DIR)
validation_meta_df = load_meta_data(VALIDATION_DIR)

In [29]:
training_meta_df.head()

,filename,label
0,a001_0_30.wav,residential_area
1,a001_120_150.wav,residential_area
2,a001_150_180.wav,residential_area
3,a001_30_60.wav,residential_area
4,a001_60_90.wav,residential_area


In [30]:
validation_meta_df.head()

,filename,label
0,178.wav,bus
1,188.wav,park
2,185.wav,forest_path
3,143.wav,beach
4,62.wav,park


In [31]:
len(training_meta_df)

1170

In [32]:
len(validation_meta_df)

390

In [33]:
training_meta_df.groupby('label').count()

,filename
label,
beach,78
bus,78
cafe/restaurant,78
car,78
city_center,78
forest_path,78
grocery_store,78
home,78
library,78


In [34]:
validation_meta_df.groupby('label').count()

,filename
label,
beach,26
bus,26
cafe/restaurant,26
car,26
city_center,26
forest_path,26
grocery_store,26
home,26
library,26
